In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Bidirectional, LSTM, TimeDistributed
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam


In [6]:
df=pd.read_csv('ner.csv')

In [7]:
df

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...
47954,Sentence: 47955,Indian border security forces are accusing the...,"['JJ', 'NN', 'NN', 'NNS', 'VBP', 'VBG', 'PRP$'...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe..."
47955,Sentence: 47956,Indian officials said no one was injured in Sa...,"['JJ', 'NNS', 'VBD', 'DT', 'NN', 'VBD', 'VBN',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
47956,Sentence: 47957,Two more landed in fields belonging to a nearb...,"['CD', 'JJR', 'VBD', 'IN', 'NNS', 'VBG', 'TO',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
47957,Sentence: 47958,They say not all of the rockets exploded upon ...,"['PRP', 'VBP', 'RB', 'DT', 'IN', 'DT', 'NNS', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [8]:
X=df['Sentence']
Y=df['Tag']

In [9]:
max_words=36000
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences=tokenizer.texts_to_sequences(X)#replace_words_with_their_indexes

In [11]:
sequences[0]

[260,
 3,
 997,
 13,
 1838,
 245,
 452,
 4,
 545,
 1,
 121,
 2,
 60,
 6,
 595,
 1,
 861,
 3,
 184,
 89,
 21,
 12,
 54]

In [12]:
X[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [13]:
X_preprocessed=pad_sequences(sequences, maxlen=110, padding='post')

In [14]:
words2id=tokenizer.word_index
id2words={}
for key, value in words2id.items():
    id2words[value]=key

In [15]:
Y[0]

"['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']"

In [16]:
type(Y[0])

str

In [17]:
from ast import literal_eval
Y_ready=[]
for sentence_tag in Y:
    Y_ready.append(literal_eval(sentence_tag))

In [18]:
Y_ready[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [19]:
type(Y_ready[0])

list

In [20]:
tags=[]
for sentence_tag in Y_ready:
    for tag in sentence_tag:
        if tag not in tags:
            tags.append(tag)

In [21]:
tags

['O',
 'B-geo',
 'B-gpe',
 'B-per',
 'I-geo',
 'B-org',
 'I-org',
 'B-tim',
 'B-art',
 'I-art',
 'I-per',
 'I-gpe',
 'I-tim',
 'B-nat',
 'B-eve',
 'I-eve',
 'I-nat']

In [23]:
num_tags=len(tags)

In [24]:
tags2id={}
for i,tag in enumerate(tags):
    tags2id[tag]=i

In [25]:
tags2id

{'O': 0,
 'B-geo': 1,
 'B-gpe': 2,
 'B-per': 3,
 'I-geo': 4,
 'B-org': 5,
 'I-org': 6,
 'B-tim': 7,
 'B-art': 8,
 'I-art': 9,
 'I-per': 10,
 'I-gpe': 11,
 'I-tim': 12,
 'B-nat': 13,
 'B-eve': 14,
 'I-eve': 15,
 'I-nat': 16}

In [26]:
id2tags={}
for key, value in tags2id.items():
    id2tags[value]=key

In [28]:
Y_preprocessed=[]
for sentence_tag in Y_ready:
    result=[]
    for tag in sentence_tag:
        result.append(tags2id[tag])
    Y_preprocessed.append(result)

In [29]:
Y_preprocessed[0]

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]

In [30]:
Y_preprocessed=pad_sequences(Y_preprocessed, maxlen=110, padding='post')

In [31]:
df.shape

(47959, 4)

In [32]:
X_train=X_preprocessed[:30000]
Y_train=Y_preprocessed[:30000]

X_val=X_preprocessed[30000:38000]
Y_val=Y_preprocessed[30000:38000]

X_test=X_preprocessed[38000:]
Y_test=Y_preprocessed[38000:]

In [33]:
train_dataset=tf.data.Dataset.from_tensor_slices((X_train, Y_train))
val_dataset=tf.data.Dataset.from_tensor_slices((X_val, Y_val))
test_dataset=tf.data.Dataset.from_tensor_slices((X_test, Y_test))



In [34]:
batch_size=128
train_dataset=train_dataset.batch(batch_size)
val_dataset=val_dataset.batch(batch_size)
test_dataset=test_dataset.batch(batch_size)

In [36]:
model=Sequential()
model.add(Embedding(max_words, 300, input_length=110))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(TimeDistributed(Dense(num_tags, activation='softmax')))

In [45]:
model.compile(loss=sparse_categorical_crossentropy,
             metrics='accuracy',
             optimizer='adam')

In [46]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
235/235 [==============================] - 75s 314ms/step - loss: 0.1927 - accuracy: 0.9656 - val_loss: 0.1149 - val_accuracy: 0.9703
Epoch 2/5
235/235 [==============================] - 73s 311ms/step - loss: 0.1031 - accuracy: 0.9718 - val_loss: 0.0925 - val_accuracy: 0.9743
Epoch 3/5
235/235 [==============================] - 73s 309ms/step - loss: 0.0746 - accuracy: 0.9770 - val_loss: 0.0738 - val_accuracy: 0.9773
Epoch 4/5
235/235 [==============================] - 72s 308ms/step - loss: 0.0590 - accuracy: 0.9805 - val_loss: 0.0689 - val_accuracy: 0.9783
Epoch 5/5
235/235 [==============================] - 73s 310ms/step - loss: 0.0490 - accuracy: 0.9833 - val_loss: 0.0699 - val_accuracy: 0.9796


In [48]:
model.evaluate(test_dataset)

78/78 [==============================] - 8s 96ms/step - loss: 0.0666 - accuracy: 0.9802


[0.06660435348749161, 0.9802207350730896]

In [ ]:
def prediction(model, test_sentence, id2tags, id2words):
    sentence=test_sentence[test_sentence>0]
    words_list=[]
    for id in list(sentence):
        words_list.append(id2words[id])
    original_sentence=' '.join(words_list)
    
    prediction=model.predict(test_sentence)
    prediction=np.argmax(prediction[0], axis=1)
    prediction=prediction[:len(original_sentence)]
    
    predicted_tags=[]
    for tag in list(prediction):
        predicted_tags.append(id2tags[id])
    return original_sentence, predicted_tags
    

In [47]:
test=X_test[0].reshape(1, 110)
model.predict(test)

1/1 [==============================] - 0s 361ms/step


array([[[9.9943060e-01, 7.3431984e-05, 4.6992303e-05, ...,
         2.0277184e-05, 2.1248827e-05, 3.5781729e-06],
        [9.9947840e-01, 9.8307581e-05, 1.4149036e-05, ...,
         1.5386371e-05, 1.3433023e-05, 2.3498844e-06],
        [9.9708986e-01, 6.1090628e-04, 4.2267133e-05, ...,
         5.8014593e-05, 4.8605169e-05, 1.1372216e-05],
        ...,
        [9.9999523e-01, 3.4427882e-07, 1.2629976e-06, ...,
         9.7068856e-08, 9.6073478e-08, 1.1344597e-08],
        [9.9998617e-01, 1.3645813e-06, 3.2161088e-06, ...,
         4.8210427e-07, 5.1817182e-07, 7.6567716e-08],
        [9.9983871e-01, 2.0322506e-05, 2.1986612e-05, ...,
         9.0879030e-06, 1.0820420e-05, 2.4572594e-06]]], dtype=float32)